<a href="https://colab.research.google.com/github/ttktjmt/mjlab_myosuite/blob/main/notebooks/train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!if [ ! -d "mjlab_myosuite" ]; then git clone -q https://github.com/ttktjmt/mjlab_myosuite.git; fi
%cd /content/mjlab_myosuite
!pip install mujoco==3.3.7 -q
!pip install "myosuite @ git+https://github.com/MyoHub/myosuite.git@mjx" -q
!pip install "mujoco-warp @ git+https://github.com/google-deepmind/mujoco_warp@486642c3fa262a989b482e0e506716d5793d61a9" -q
!pip install -e . -q

/content/mjlab_myosuite
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 18.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getti

In [2]:
import os
from google.colab import userdata

try:
    os.environ['WANDB_API_KEY'] = userdata.get('WANDB_API_KEY')
    os.environ['WANDB_ENTITY'] = userdata.get('WANDB_ENTITY')
except:
    os.environ['WANDB_MODE'] = 'disabled'

In [5]:
!python scripts/train.py Mjlab-MyoSuite-myoElbowPose1D6MRandom-v0 --env.num_envs 4

Streaming output truncated to the last 5000 lines.
--------------------------------------------------------------------------------
                         Iteration time: 0.23s
                           Time elapsed: 00:02:53
                                    ETA: 00:01:06

################################################################################
                          Learning iteration 723/1000                            

                            Total steps: 69504 
                       Steps per second: 426 
                        Collection time: 0.113s 
                          Learning time: 0.111s 
                        Mean value loss: 755.1472
                    Mean surrogate loss: -0.0065
                      Mean entropy loss: 8.5730
                            Mean reward: 15.87
                    Mean episode length: 26.75
                  Mean action noise std: 1.01
-----------------------------------------------------------------------------

In [6]:
import os
import re
from pathlib import Path

# Find the most recent training run
log_dir = Path("/content/mjlab_myosuite/logs/rsl_rl/cartpole")
if log_dir.exists():
  runs = sorted(log_dir.glob("*"), key=os.path.getmtime, reverse=True)
  if runs:
    latest_run = runs[0]
    print(f"✓ Latest training run: {latest_run.name}\n")

    # List checkpoints - sorted by iteration number
    checkpoints = list(latest_run.glob("model_*.pt"))
    if checkpoints:
      # Extract iteration number and sort numerically
      def get_iteration(ckpt):
        match = re.search(r"model_(\d+)\.pt", ckpt.name)
        return int(match.group(1)) if match else 0

      checkpoints = sorted(checkpoints, key=get_iteration)

      print(f"Found {len(checkpoints)} checkpoints:")
      for ckpt in checkpoints[-5:]:  # Show last 5
        size_mb = ckpt.stat().st_size / (1024 * 1024)
        iteration = get_iteration(ckpt)
        print(f"  • {ckpt.name} (iteration {iteration}, {size_mb:.2f} MB)")

      # Store the last checkpoint path
      last_checkpoint = str(checkpoints[-1])
      print(f"\n💾 Last checkpoint: {last_checkpoint}")
    else:
      print("⚠ No checkpoints found yet")
  else:
    print("⚠ No training runs found")
else:
  print("⚠ Log directory not found. Have you run training yet?")

⚠ Log directory not found. Have you run training yet?


In [8]:
import subprocess
import sys
import re

WANDB_RUN_ID = "q47fv4j1"

process = subprocess.Popen(
  [
    "python",
    "scripts/play.py",
    "Mjlab-MyoSuite-myoElbowPose1D6MRandom-v0",
    # "--checkpoint_file",
    # last_checkpoint,
    "--wandb-run-path",
    f"{userdata.get('WANDB_ENTITY')}/mjlab_myosuite/{WANDB_RUN_ID}",
    "--num_envs",
    "4",
  ],
  stdout=subprocess.PIPE,
  stderr=subprocess.STDOUT,
  universal_newlines=True,
  bufsize=1,
)

detected_port = None

for line in process.stdout:
  print(line, end="")
  sys.stdout.flush()

  # Extract port number from viser output
  port_match = re.search(r":(\d{4})", line)
  if port_match and "viser" in line.lower():
    detected_port = int(port_match.group(1))
    print("\n" + "=" * 34)
    print(f"✅ Server is running on port {detected_port}!")
    print("=" * 34)
    break

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoSarcArmReachFixed-v0 already in registry.
  logger.warn(f"Overriding environment {new_spec.id} already in registry.")
/usr/local/lib/python3.12/dist-packages/gymnasium/envs/registration.py:636: UserWarning: WARN: Overriding environment myoFatiArmReachFixed-v0 already in registry.


In [9]:
from google.colab import output

port = detected_port if detected_port else 8081
output.serve_kernel_port_as_iframe(port=port, height=700)

<IPython.core.display.Javascript object>